In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence
import pickle

In [3]:
# Load dataset (First 700 rows)
data = pd.read_csv("Roman-Urdu-Poetry.csv").iloc[:700]
poetry_lines = data["Poetry"].dropna().tolist()

# Prepare Text Encoding
text = " ".join(poetry_lines)
words = text.split()
word_encoder = LabelEncoder()
word_encoder.fit(words)

word_to_index = {word: i for i, word in enumerate(word_encoder.classes_)}
index_to_word = {i: word for word, i in word_to_index.items()}


In [4]:
# Convert text to sequences
sequences = []
for i in range(len(words) - 5):
    sequences.append([word_to_index[word] for word in words[i : i + 6]])

sequences = np.array(sequences)
X, y = sequences[:, :-1], sequences[:, -1]

# Convert sequences to PyTorch tensors and pad
X = [torch.tensor(seq, dtype=torch.long) for seq in X]
X = pad_sequence(X, batch_first=True, padding_value=0)
y = torch.tensor(y, dtype=torch.long)

dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [5]:
# Define the GRU Model
class GRUModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim=50, hidden_dim=128, output_dim=None):
        super(GRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 128)
        self.fc2 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.gru(x)
        x = self.relu(self.fc1(x[:, -1, :]))  # Use last hidden state
        x = self.log_softmax(self.fc2(x))
        return x

In [6]:
# Initialize model
vocab_size = len(word_to_index)
model = GRUModel(vocab_size=vocab_size, output_dim=vocab_size)

# Initialize weights
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
    elif isinstance(m, nn.GRU):
        for name, param in m.named_parameters():
            if "weight" in name:
                nn.init.xavier_uniform_(param)

model.apply(init_weights)

criterion = nn.NLLLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GRUModel(
  (embedding): Embedding(11044, 50)
  (gru): GRU(50, 128, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=11044, bias=True)
  (relu): ReLU()
  (log_softmax): LogSoftmax(dim=1)
)

In [7]:
# Train Model
epochs = 55
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in dataloader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()
        output = model(batch_X)
        loss = criterion(output, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(dataloader):.4f}")


Epoch 1/55, Loss: 6.9514
Epoch 2/55, Loss: 6.5218
Epoch 3/55, Loss: 6.1905
Epoch 4/55, Loss: 5.8602
Epoch 5/55, Loss: 5.5436
Epoch 6/55, Loss: 5.2198
Epoch 7/55, Loss: 4.8696
Epoch 8/55, Loss: 4.4777
Epoch 9/55, Loss: 4.0753
Epoch 10/55, Loss: 3.7139
Epoch 11/55, Loss: 3.4158
Epoch 12/55, Loss: 3.1720
Epoch 13/55, Loss: 2.9578
Epoch 14/55, Loss: 2.7734
Epoch 15/55, Loss: 2.6018
Epoch 16/55, Loss: 2.4436
Epoch 17/55, Loss: 2.2983
Epoch 18/55, Loss: 2.1616
Epoch 19/55, Loss: 2.0334
Epoch 20/55, Loss: 1.9081
Epoch 21/55, Loss: 1.7955
Epoch 22/55, Loss: 1.6863
Epoch 23/55, Loss: 1.5849
Epoch 24/55, Loss: 1.4891
Epoch 25/55, Loss: 1.3981
Epoch 26/55, Loss: 1.3115
Epoch 27/55, Loss: 1.2331
Epoch 28/55, Loss: 1.1513
Epoch 29/55, Loss: 1.0848
Epoch 30/55, Loss: 1.0179
Epoch 31/55, Loss: 0.9562
Epoch 32/55, Loss: 0.8948
Epoch 33/55, Loss: 0.8409
Epoch 34/55, Loss: 0.7884
Epoch 35/55, Loss: 0.7429
Epoch 36/55, Loss: 0.6938
Epoch 37/55, Loss: 0.6559
Epoch 38/55, Loss: 0.6192
Epoch 39/55, Loss: 0.

In [8]:
# Save Model
torch.save(model.state_dict(), "Roman_Urdu_model.pth")
torch.save(optimizer.state_dict(), "optimizer.pth")
# Save Training History
with open("training_history.pkl", "wb") as f:
    pickle.dump({'loss': total_loss}, f)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')